Almacenamiento y carga de modelos --- 0:00 min
===

* Última modificación: Marzo 7, 2022 | YouTube

Importación de librerías
---

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

In [2]:
!pip3 install --quiet pyyaml h5py

Dataset
--

In [3]:
(train_images, train_labels), (
    test_images,
    test_labels,
) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

Definición del modelo
--

In [4]:
def create_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Dense(512, activation="relu", input_shape=(784,)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()],
    )

    return model


model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Almacenamiento de checkpoints durante el entrenamiento
--

**Checkpoint callback**

In [5]:
checkpoint_path = "/tmp/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
)

model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback],
)

Epoch 1/10
16/32 [==============>...............] - ETA: 0s - loss: 1.5381 - sparse_categorical_accuracy: 0.5508
Epoch 1: saving model to /tmp/training_1/cp.ckpt
32/32 [==============================] - 1s 10ms/step - loss: 1.1611 - sparse_categorical_accuracy: 0.6620 - val_loss: 0.7098 - val_sparse_categorical_accuracy: 0.7690
Epoch 2/10
19/32 [================>.............] - ETA: 0s - loss: 0.4294 - sparse_categorical_accuracy: 0.8668
Epoch 2: saving model to /tmp/training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4251 - sparse_categorical_accuracy: 0.8810 - val_loss: 0.5491 - val_sparse_categorical_accuracy: 0.8240
Epoch 3/10
18/32 [===============>..............] - ETA: 0s - loss: 0.2984 - sparse_categorical_accuracy: 0.9167
Epoch 3: saving model to /tmp/training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.2940 - sparse_categorical_accuracy: 0.9240 - val_loss: 0.4924 - val_sparse_categorical_accuracy: 0.8470
Epoch 4/10
1

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [7]:
model = create_model()

loss, acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2,
)

print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3158 - sparse_categorical_accuracy: 0.1040 - 147ms/epoch - 5ms/step
Untrained model, accuracy: 10.40%


In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2,
)

print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4131 - sparse_categorical_accuracy: 0.8720 - 69ms/epoch - 2ms/step
Restored model, accuracy: 87.20%


**Opciones**

In [9]:
checkpoint_path = "/tmp/training_2/cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5 * batch_size,
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=50,
    batch_size=batch_size,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0,
)


Epoch 5: saving model to /tmp/training_2/cp-0005.ckpt

Epoch 10: saving model to /tmp/training_2/cp-0010.ckpt

Epoch 15: saving model to /tmp/training_2/cp-0015.ckpt

Epoch 20: saving model to /tmp/training_2/cp-0020.ckpt

Epoch 25: saving model to /tmp/training_2/cp-0025.ckpt

Epoch 30: saving model to /tmp/training_2/cp-0030.ckpt

Epoch 35: saving model to /tmp/training_2/cp-0035.ckpt

Epoch 40: saving model to /tmp/training_2/cp-0040.ckpt

Epoch 45: saving model to /tmp/training_2/cp-0045.ckpt

Epoch 50: saving model to /tmp/training_2/cp-0050.ckpt


In [10]:
!ls -1 {checkpoint_dir}

checkpoint
cp-0000.ckpt.data-00000-of-00001
cp-0000.ckpt.index
cp-0005.ckpt.data-00000-of-00001
cp-0005.ckpt.index
cp-0010.ckpt.data-00000-of-00001
cp-0010.ckpt.index
cp-0015.ckpt.data-00000-of-00001
cp-0015.ckpt.index
cp-0020.ckpt.data-00000-of-00001
cp-0020.ckpt.index
cp-0025.ckpt.data-00000-of-00001
cp-0025.ckpt.index
cp-0030.ckpt.data-00000-of-00001
cp-0030.ckpt.index
cp-0035.ckpt.data-00000-of-00001
cp-0035.ckpt.index
cp-0040.ckpt.data-00000-of-00001
cp-0040.ckpt.index
cp-0045.ckpt.data-00000-of-00001
cp-0045.ckpt.index
cp-0050.ckpt.data-00000-of-00001
cp-0050.ckpt.index


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/tmp/training_2/cp-0050.ckpt'

In [12]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2,
)

print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4824 - sparse_categorical_accuracy: 0.8800 - 146ms/epoch - 5ms/step
Restored model, accuracy: 88.00%


Almacenamiento manual de los pesos
--

In [13]:
model.save_weights("/tmp/checkpoints/my_checkpoint")

model = create_model()

model.load_weights("/tmp/checkpoints/my_checkpoint")

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4824 - sparse_categorical_accuracy: 0.8800 - 143ms/epoch - 4ms/step
Restored model, accuracy: 88.00%


Almacenamiento del modelo completo
---

**Formato SavedModel**

In [14]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p /tmp/saved_model
model.save("/tmp/saved_model/my_model/")

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.2063 - sparse_categorical_accuracy: 0.6720
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4161 - sparse_categorical_accuracy: 0.8890
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.3037 - sparse_categorical_accuracy: 0.9170
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2115 - sparse_categorical_accuracy: 0.9390
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1495 - sparse_categorical_accuracy: 0.9700
INFO:tensorflow:Assets written to: /tmp/saved_model/my_model/assets


In [15]:
!ls /tmp/saved_model

my_model


In [16]:
!ls -1 /tmp/saved_model/my_model

assets
keras_metadata.pb
saved_model.pb
variables


In [17]:
new_model = tf.keras.models.load_model("/tmp/saved_model/my_model/")

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
loss, acc = new_model.evaluate(
    test_images,
    test_labels,
    verbose=2,
)

print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4529 - sparse_categorical_accuracy: 0.8520 - 150ms/epoch - 5ms/step
Restored model, accuracy: 85.20%
(1000, 10)


**Formato HDF5**

In [19]:
model = create_model()
model.fit(
    train_images,
    train_labels,
    epochs=5,
)
model.save("/tmp/my_model.h5")

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1487 - sparse_categorical_accuracy: 0.6820
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4128 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2748 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1979 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1412 - sparse_categorical_accuracy: 0.9690


In [20]:
new_model = tf.keras.models.load_model("/tmp/my_model.h5")
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
loss, acc = new_model.evaluate(
    test_images,
    test_labels,
    verbose=2,
)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4456 - sparse_categorical_accuracy: 0.8580 - 144ms/epoch - 4ms/step
Restored model, accuracy: 85.80%
